In [1]:

# ================================================
# 1️⃣ Env Prepare Install Required Packages
# ================================================

#%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo 

Using Python 3.12.12 environment at: /home/ospost/micromamba/envs/ML
Resolved 5 packages in 17ms                                          
Audited 5 packages in 0.15ms


In [2]:
import torch
import bitsandbytes as bnb
from collections import Counter
from unsloth import FastLanguageModel


model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/gpt-oss-20b",
#    load_in_4bit=True,
)

bnb_layers = []
for name, module in model.named_modules():
    if isinstance(module, bnb.nn.Linear4bit):
        bnb_layers.append(name)

print(f"BNB 4-bit layers found: {len(bnb_layers)}")
print("Sample BNB layer:", bnb_layers[0] if bnb_layers else "❌ NONE")
print("Model compute dtype:", model.dtype)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ospost/micromamba/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    GRID A100D-40C. Num GPUs = 1. Max memory: 39.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1. CUDA: 8.0. CUDA Toolkit: 12.9. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.20s/it]


BNB 4-bit layers found: 1632
Sample BNB layer: model.layers.0.self_attn.q_proj
Model compute dtype: torch.bfloat16


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    checkpoint_path="ospost/gpt-oss-20b-sft-adapter"
)


Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
assert any("lora_" in n for n,_ in model.named_parameters())

lora_params = [
    (n, p) for n, p in model.named_parameters() if "lora_" in n
]

print(f"LoRA parameters found: {len(lora_params)}")
print("Sample LoRA param:", lora_params[0][0] if lora_params else "❌ NONE")

print("LoRA dtypes:", Counter(p.dtype for _, p in lora_params))

trainable = [
    n for n, p in model.named_parameters() if p.requires_grad
]

print(f"Trainable params count: {len(trainable)}")
print("Sample trainable param:", trainable[0] if trainable else "❌ NONE")



LoRA parameters found: 192
Sample LoRA param: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
LoRA dtypes: Counter({torch.float32: 192})
Trainable params count: 192
Sample trainable param: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight


(ML) ospost@A100:~/workspace/llama.cpp$ cp -fr build/bin/* ./

In [5]:
import os
os.environ["LD_LIBRARY_PATH"] = os.path.abspath("../llama.cpp") + ":" + os.environ.get("LD_LIBRARY_PATH", "")
# Optional: verify
print(os.environ["LD_LIBRARY_PATH"])

/home/ospost/workspace/llama.cpp:


In [6]:
import subprocess

res = subprocess.run(["../llama.cpp/llama-quantize", "--help"], capture_output=True, text=True)
print("Return code:", res.returncode)
print("Stdout:", res.stdout[:500])
print("Stderr:", res.stderr[:500])


Return code: 1
Stdout: usage: ../llama.cpp/llama-quantize [--help] [--allow-requantize] [--leave-output-tensor] [--pure] [--imatrix] [--include-weights]
       [--exclude-weights] [--output-tensor-type] [--token-embedding-type] [--tensor-type] [--prune-layers] [--keep-split] [--override-kv]
       model-f32.gguf [model-quant.gguf] type [nthreads]

  --allow-requantize: Allows requantizing tensors that have already been quantized. Warning: This can severely reduce quality compared to quantizing from 16bit or 32bit
  --
Stderr: 


In [7]:
from unsloth_zoo.llama_cpp import check_llama_cpp

quantizer, converter = check_llama_cpp("../llama.cpp")
print("Quantizer path:", quantizer)
print("Converter path:", converter)


Quantizer path: ../llama.cpp/llama-quantize
Converter path: ../llama.cpp/convert_hf_to_gguf.py


In [11]:

model.save_pretrained_merged("gpt-oss-20b-sft-mxfp4", tokenizer, save_method="mxfp4")

Found HuggingFace hub cache directory: /home/ospost/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.99s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into mxfp4: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.93s/it]


Unsloth: Merge process complete. Saved to `/home/ospost/workspace/gpt-oss-20b-fine-tune/gpt-oss-20b-sft-mxfp4`


In [12]:
!ls -lah merged_model

ls: cannot access 'merged_model': No such file or directory


In [13]:
!ls -lah gpt-oss-20b-sft-mxfp4

total 13G
drwxrwxr-x 3 ospost ospost 4.0K Jan  9 15:14 .
drwxrwxr-x 9 ospost ospost 4.0K Jan  9 15:14 ..
drwxrwxr-x 3 ospost ospost 4.0K Jan  9 15:14 .cache
-rw-rw-r-- 1 ospost ospost  15K Jan  9 15:14 chat_template.jinja
-rw-rw-r-- 1 ospost ospost 1.8K Jan  9 15:14 config.json
-rw-rw-r-- 1 ospost ospost 4.5G Jan  9 15:14 model-00000-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost 4.5G Jan  9 15:14 model-00001-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost 3.9G Jan  9 15:14 model-00002-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost  36K Jan  9 15:14 model.safetensors.index.json
-rw-rw-r-- 1 ospost ospost  446 Jan  9 15:14 special_tokens_map.json
-rw-rw-r-- 1 ospost ospost  20K Jan  9 15:14 tokenizer_config.json
-rw-rw-r-- 1 ospost ospost  27M Jan  9 15:14 tokenizer.json


To save your fine-tuned model, you can export your fine-tuned model both in bf16 format , with our on-demand dequantization of MXFP4 base models using save_method="merged_16bit"or in native MXFP4 Safetensors format using save_method="mxfp4" .

The MXFP4 native merge format offers significant performance improvements compared to the bf16 format: it uses up to 75% less disk space, reduces VRAM consumption by 50%, accelerates merging by 5-10x, and enables much faster conversion to GGUF format.

Convert the MXFP4 merged model:
python3 llama.cpp/convert_hf_to_gguf.py gpt-oss-finetuned-merged/ --outfile gpt-oss-finetuned-mxfp4.gguf


In [18]:
!ls ../llama.cpp

AGENTS.md		       llama-lookup-stats
AUTHORS			       llama-minicpmv-cli
benches			       llama-mtmd-cli
build			       llama-parallel
build-xcframework.sh	       llama-passkey
ci			       llama-perplexity
CLAUDE.md		       llama-q8dot
cmake			       llama-quantize
CMakeLists.txt		       llama-qwen2vl-cli
CMakePresets.json	       llama-retrieval
CODEOWNERS		       llama-save-load-state
common			       llama-server
CONTRIBUTING.md		       llama-simple
convert_hf_to_gguf.py	       llama-simple-chat
convert_hf_to_gguf_update.py   llama-speculative
convert_llama_ggml_to_gguf.py  llama-speculative-simple
convert_lora_to_gguf.py        llama-tokenize
docs			       llama-tts
examples		       llama-vdot
flake.lock		       Makefile
flake.nix		       media
ggml			       models
gguf-py			       mypy.ini
grammars		       pocs
include			       poetry.lock
libggml-base.so		       pyproject.toml
libggml-base.so.0	       pyrightconfig.json
libggml-base.so.0.9.5	       quantize
libggml-cpu.so		       R

In [ ]:


model.save_pretrained_gguf("gpt-oss-20b-sft-mxfp4", tokenizer, quantization_method="q4_k_m")

Unsloth: Merging model weights to mxfp4 format...
Found HuggingFace hub cache directory: /home/ospost/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|█████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 38014.84it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into mxfp4: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.56s/it]


Unsloth: Merge process complete. Saved to `/home/ospost/workspace/gpt-oss-20b-fine-tune/gpt-oss-20b-sft-mxfp4`
Unsloth: Converting to GGUF format...
Unsloth: GPT-OSS model detected - using special conversion settings
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF  might take 3 minutes.
\        /    [2] Converting GGUF  to ['None'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into  GGUF format.
This might take 3 minutes...
